In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [3]:
#提供环境
conf = SparkConf().setAppName("Spark Read MySQL").set("spark.jars", "/data/lab/mysql-connector-j-8.4.0.jar")
spark = SparkSession.builder.config(conf=conf).getOrCreate()
silver_path = "silver_data.csv"
df = spark.read.csv(silver_path, header=True, inferSchema=True)

25/04/12 08:05:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/12 08:05:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/12 08:05:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/04/12 08:05:20 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/04/12 08:05:20 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/04/12 08:05:20 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/04/12 08:05:20 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/04/12 08:05:20 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/04/12 08:05:20 WARN Utils: Serv

In [4]:
##向数据库写入数据
df.write.format("jdbc") \
        .option("url", "jdbc:mysql://localhost:3306/spark_project?useSSL=false&serverTimezone=UTC") \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .option("dbtable", "silver") \
        .option("user", "root") \
        .option("password", "123456") \
        .mode("append") \
        .save()

25/04/12 08:08:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

In [5]:
##验证是否写入数据
jdbc_df = spark.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/spark_project?useSSL=false&serverTimezone=UTC") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "silver") \
    .option("user", "root") \
    .option("password", "123456") \
    .load()

jdbc_df.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|seniorCitizen|partner|dependents|tenure|phoneService|   multipleLines|internetService|onlineSecurity|onlineBackup|deviceProtection|techSupport|streamingTV|streamingMovies|      contract|paperlessBilling|       paymentMethod|monthlyCharges|totalCharges|churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|          0.0|    Yes|        No|   1.0|          No|No phone service|            DSL|            No|         Yes|              No|         No|    

In [8]:
##通过临时视图简略地复现我所完成的LLM代码
df.createOrReplaceTempView("silver")
select_months = spark.sql("""
SELECT customerID, tenure
FROM silver
WHERE tenure > 24 AND tenure < 60;
""")
select_months.show(5)
##这里LLM没有理解我的指令，因此没有取闭区间

+----------+------+
|customerID|tenure|
+----------+------+
|7892-POOKP|  28.0|
|0280-XJGEX|  49.0|
|5129-JLPIS|  25.0|
|6322-HRPFA|  49.0|
|6865-JZNKO|  30.0|
+----------+------+
only showing top 5 rows



In [9]:
##正确情况
select_months_true = spark.sql("""
SELECT customerID, tenure
FROM silver
WHERE tenure BETWEEN 24 AND 60;
""")
select_months_true.show(5)

+----------+------+
|customerID|tenure|
+----------+------+
|7892-POOKP|  28.0|
|0280-XJGEX|  49.0|
|5129-JLPIS|  25.0|
|6322-HRPFA|  49.0|
|6865-JZNKO|  30.0|
+----------+------+
only showing top 5 rows

